In [2]:
import pandas as pd
import math

In [3]:
df = pd.read_csv("a_d_counted_df_labelled.csv")

In [5]:
# Initialize the dictionary with each key as "INPUT|CONTROLLER|x_a" where x_a is activation type, and value being a list of 
# [num_mentions, pos_or_neg]

interaction_dict = {}
i = 0
for i in range(0, len(df.INPUT)):
    if df.NUM_LABEL[i] == 1:
        interaction_dict[f"{df.INPUT[i]}|{df.CONTROLLER[i]}|pos"] = [df.COUNTER[i], df.NUM_LABEL[i]]
    elif df.NUM_LABEL[i] == -1:
        interaction_dict[f"{df.OUTPUT[i]}|{df.CONTROLLER[i]}|neg"] = [df.COUNTER[i], df.NUM_LABEL[i]]
    i += 1

In [5]:
'''
### Artifact that may be needed in the future ###

Kind of forgot what this does...

If the input has a suffix and the base input exists as a key, then we add the count to the base input i.
If the input has a suffix and the base input does not exist as a key, then we add it to temp_dict to be later updated into the interaction_dict.
When added to temp, the num_label is inverted. Note that this code works only for activations as of now.
'''
'''
temp_dict = {}

for key in interaction_dict:
    input_met, controller_met = key.split("|")
    if input_met[-2] == ".":
        base_reconstructed = f"{input_met[:-2]}|{controller_met}"
        if interaction_dict[key][1] == 1:
            if not base_reconstructed in interaction_dict:
                temp_dict[base_reconstructed] = [interaction_dict[key][0], -interaction_dict[key][1]]
            else:
                interaction_dict[base_reconstructed][0] += interaction_dict[key][0]
        if interaction_dict[key][1] == -1:
            if not base_reconstructed in interaction_dict:
                temp_dict[base_reconstructed] = [interaction_dict[key][0], -interaction_dict[key][1]]
            else:
                interaction_dict[base_reconstructed][0] += interaction_dict[key][0]

interaction_dict.update(temp_dict)
'''

'\ntemp_dict = {}\n\nfor key in interaction_dict:\n    input_met, controller_met = key.split("|")\n    if input_met[-2] == ".":\n        base_reconstructed = f"{input_met[:-2]}|{controller_met}"\n        if interaction_dict[key][1] == 1:\n            if not base_reconstructed in interaction_dict:\n                temp_dict[base_reconstructed] = [interaction_dict[key][0], -interaction_dict[key][1]]\n            else:\n                interaction_dict[base_reconstructed][0] += interaction_dict[key][0]\n        if interaction_dict[key][1] == -1:\n            if not base_reconstructed in interaction_dict:\n                temp_dict[base_reconstructed] = [interaction_dict[key][0], -interaction_dict[key][1]]\n            else:\n                interaction_dict[base_reconstructed][0] += interaction_dict[key][0]\n\ninteraction_dict.update(temp_dict)\n'

In [ ]:
# Finds the number of values in interaction_dict that have contradictory findings.
# At the very end output, the data "loss" is from the contradictory findings being squished into the proportion.

'''
# This is a test dictionary artifact.
interaction_dict = {
    "AnInput1|AController1|pos": [3, 1],
    "AnInput2|AController2|neg": [3, 1],
    "AnInput3|AController3|pos": [3, 1],
    "AnInput1|AController1|neg": [3, 1]
}
'''

counter = 0

for key_outer in interaction_dict:
    split_outer = key_outer.split("|")
    pos_outer = True

    if split_outer[2] == "neg":
        pos_outer = False

    for key_inner in interaction_dict:
        split_inner = key_inner.split("|")
        pos_inner = True
        
        if split_inner[2] == "neg":
            pos_inner = False

        if key_inner[:-3] == key_outer[:-3] and split_outer[2] != split_inner[2]:
            # print(f"{key_inner[:-3]} has activation type: {split_inner[2]}")
            # print(f"{key_outer[:-3]} has activation type: {split_outer[2]}")
            counter += 1

# Divide this by 2 because each hit will read over itself again.
print(counter/2)

In [7]:
# We make a proportion dictionary with each key as "Input/Output|Controller" and each value as [pos_count, total_count]

# initialize the dictionary
prop_dict = {}

for key in interaction_dict:
    prop_dict[key[:-4]] = [0, 0]


In [11]:
# Add just Input/Output|Controller as key to prop_dict, and if the activation is postive, add to the first element in the list value. Otherwise
# just add to total.

for key in interaction_dict:
    split_key = key.split("|")
    count = interaction_dict[key][0]
    if split_key[2] == "pos":
        prop_dict[key[:-4]][0] += count
        prop_dict[key[:-4]][1] += count
    if split_key[2] == "neg":
        prop_dict[key[:-4]][1] += count

In [12]:
# for each entry in prop_dict, divide pos by total to get the proportion. Then input into the modified tanh function to get our edge value.
# subtract 1 for tanh, leave -1 out for sigmoid


for key in prop_dict:
    proportion = prop_dict[key][0] / prop_dict[key][1]
    y = (1/(1+math.exp(-5*(proportion-.5))))
    prop_dict[key].append(y)

In [13]:
# Change dictionary's form into dataframe-compatible form.
df_dict = {
    "INPUT": [],
    "CONTROLLER": [],
    "EDGE": [],
    "POS": [],
    "TOTAL": [],
}
for key in prop_dict:
    met_input, met_cont = key.split("|")
    df_dict["INPUT"].append(met_input)
    df_dict["CONTROLLER"].append(met_cont)
    df_dict["EDGE"].append(prop_dict[key][2])
    df_dict["POS"].append(prop_dict[key][0])
    df_dict["TOTAL"].append(prop_dict[key][1])


In [14]:
prop_df = pd.DataFrame.from_dict(df_dict)

In [15]:
prop_df

,INPUT,CONTROLLER,EDGE,POS,TOTAL
0,apoptosis::go:GO:0006915,ROS::chebi:CHEBI:26523,0.897719,114,122
1,inflammation::go:GO:0006954,LPS::uniprot:O14896,0.851357,90,106
2,insulin secretion::go:GO:0030073,glucose::chebi:CHEBI:17234,0.836801,86,104
3,FBS::uniprot:Q9HAH7,heat::pfam:PF02985,0.075858,0,72
4,cell death::go:GO:0008219,ROS::chebi:CHEBI:26523,0.902227,68,72
...,...,...,...,...,...
98148,‐8::uaz:UAZE2809038,mycoplasma infection::mesh:D009175,0.924142,2,2
98149,′-CTGCTGGATGACGTGAGGT AACC-3′::uaz:UAZE280B22D...,probe (5′-VIC TGTCTCGCTCCGTGGCCTTAGCTG-3′::uaz...,0.075858,0,2
98150,’s::uaz:UAZE2809973,MVs::pubchem:449,0.924142,2,2
98151,♂::uaz:UAZE29982,cecropin::pfam:PF00272,0.924142,2,2


In [21]:
prop_df.to_csv("all_tanh.csv")